In [10]:
import librosa
import numpy as np
import json

In [24]:
DATA_PATH = 'data.json'

with open(DATA_PATH, "r") as fp:
    data = json.load(fp)

chroma = data["chroma_vector"]
labels = data["labels"]

In [30]:
max_length = max(len(arr) for arr in chroma)

chroma = [
    np.pad(
        arr,
        pad_width=((0, max_length - len(arr)), (0, 0)),  
        mode='constant',
        constant_values=0
    )
    for arr in chroma
]

chroma = np.array(chroma)
print(chroma.shape)

labels = np.array(labels)
print(labels.shape)

(1996, 290, 12)

In [32]:
from sklearn.model_selection import train_test_split

def split_data(features, labels):
  """
  split_data(features, labels) splits the input data (features, labels) into training
  set, validation set, and testing set split as 60-10-30 i.e. 60% of the input data
  makes up the training set, 10% makes up the validation set, and 30% makes up the
  testing set.
  """
  
  X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.30, random_state = 42)
  X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.15, random_state = 42)
  return X_train, X_test, X_val, y_train, y_test, y_val

In [ ]:
X_train, X_test, X_validation, y_train, y_test, y_validation = split_data(chroma, labels)

In [31]:
import tensorflow.keras as keras

model = keras.Sequential()

model.add(keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = input_shape))
model.add(keras.layers.MaxPool2D((3, 3), strides = (2, 2), padding = 'same'))
model.add(keras.layers.BatchNormalization())

# Second Convolution Layer
model.add(keras.layers.Conv2D(64, (3, 3), activation = 'relu', input_shape = input_shape))
model.add(keras.layers.MaxPool2D((3, 3), strides = (2, 2), padding = 'same'))
model.add(keras.layers.BatchNormalization())

# Third Convolution Layer
model.add(keras.layers.Conv2D(128, (2, 2), activation = 'relu', input_shape = input_shape))
model.add(keras.layers.MaxPool2D((2, 2), strides = (2, 2), padding = 'same'))
model.add(keras.layers.BatchNormalization())

# Converting the neurons into a 1D layer
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation = 'relu'))
model.add(keras.layers.Dropout(0.3))

  # Condensing neuron layer into 10 nodes, each corresponding to a music genre label
model.add(keras.layers.Dense(10, activation = 'softmax'))

(1996,)